# Welcome To the ONNX Runtime Tutorial!

This notebook is designed to demonstrate the features of the ONNXRT passes integrated into MASE as part of the MASERT framework.

## Section 1. ONNX Runtime Optimizations
Firstly, we will show you how we can utilise the ONNX RT optimizations. We expect to see a speed up without a loss in model accuracy. We will use a simple model, `jsc-toy`, and compare the optimized model to the original model using the `Machop API`.

First, we load the machop requirements by running the cell below.

In [2]:
import sys
import os
from pathlib import Path
import toml

# Figure out the correct path
machop_path = Path(".").resolve().parent.parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

# Add directory to the PATH so that chop can be called
new_path = "../../../machop"
full_path = os.path.abspath(new_path)
os.environ['PATH'] += os.pathsep + full_path

from chop.tools.utils import to_numpy_if_tensor
from chop.tools.logger import set_logging_verbosity
from chop.tools import get_cf_args, get_dummy_input
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph
from chop.models import get_model_info, get_model, get_tokenizer
from chop.dataset import MaseDataModule, get_dataset_info
from chop.passes.graph.transforms import metadata_value_type_cast_transform_pass
from chop.passes.graph import (
    summarize_quantization_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    onnx_runtime_transform_pass,
    runtime_analysis_pass,
    )

set_logging_verbosity("info")

/root/anaconda3/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-03-20 18:29:58,588] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO     Set logging level to info
I0320 18:30:00.082091 140258125256512 logger.py:44] Set logging level to info


We then load in a demonstration toml file and set the relevant pass arguments (this is all done automatically if we were to use the command line, see [Section X]())

In [2]:
JSC_TOML_PATH = "../../../machop/configs/onnx/jsc_gpu_ort.toml"

# Reading TOML file and converting it into a Python dictionary
with open(JSC_TOML_PATH, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt' section and its children
onnx_config = pass_args.get('passes', {}).get('onnxruntime', {})
# Extract the 'passes.runtime_analysis' section and its children
runtime_analysis_config = pass_args.get('passes', {}).get('runtime_analysis', {})

# Load the basics in
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)

data_module.prepare_data()
data_module.setup()

# Add the data_module and other necessary information to the configs
configs = [onnx_config, runtime_analysis_config]
for config in configs:
    config['task'] = pass_args['task']
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['dataset'] = pass_args['dataset']
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

Next, we train the `jsc-toy` model using the machop `train` action with the config from the toml file. You may want to switch to GPU for this task - it will not affect the cpu optimizations later on.

In [3]:
# !ch train --config {JSC_TOML_PATH} --accelerator gpu

Then we load in the checkpoint. You will have to adjust this according to where it has been stored in the mase_output directory.

In [4]:
# Load in the trained checkpoint - change this accordingly
JSC_CHECKPOINT_PATH = "../../../mase_output/jsc-toy_cls_jsc/software/training_ckpts/best.ckpt"

model = load_model(load_name=JSC_CHECKPOINT_PATH, load_type="pl", model=model)

# Initiate metadata
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)
mg, _ = init_metadata_analysis_pass(mg, None)

# Copy original graph for analysis later
mg_original = deepcopy_mase_graph(mg)

mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)
mg, _ = metadata_value_type_cast_transform_pass(mg, pass_args={"fn": to_numpy_if_tensor})

INFO     Loaded pytorch lightning checkpoint from ../../../mase_output/jsc-toy_cls_jsc/software/training_ckpts/best.ckpt
I0320 18:10:46.938912 140225141540672 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from ../../../mase_output/jsc-toy_cls_jsc/software/training_ckpts/best.ckpt


We then run the `onnx_runtime_transform_pass` which completes the optimizations using the dataloader and `jsc-toy` model. This returns metadata containing the paths to the models:

- `onnx_path` (the optimized model)
- `onnx_dynamic_quantized_path` (the dynamically )

In this case, since we are not quantizing the model, only the `onnx_path` is available. 

The models are also stored in the directory:
```
mase_output
└── onnxrt
    └── model_task_dataset_date
        ├── optimized
        ├── pre_processed
        ├── static_quantized
        └── dynamic_quantized
```

In [5]:
mg, onnx_meta = onnx_runtime_transform_pass(mg, pass_args=onnx_config)

INFO     Converting PyTorch model to ONNX...
I0320 18:10:47.354932 140225141540672 onnx_runtime.py:48] Converting PyTorch model to ONNX...
INFO     Project will be created at /root/mase/mase_output/onnxrt/jsc-toy_cls_jsc_2024-03-20
I0320 18:10:47.357415 140225141540672 onnx_runtime.py:50] Project will be created at /root/mase/mase_output/onnxrt/jsc-toy_cls_jsc_2024-03-20
INFO     ONNX Conversion Complete. Stored ONNX model to /root/mase/mase_output/onnxrt/jsc-toy_cls_jsc_2024-03-20/optimized/version_14/model.onnx
I0320 18:10:47.671911 140225141540672 onnx_runtime.py:68] ONNX Conversion Complete. Stored ONNX model to /root/mase/mase_output/onnxrt/jsc-toy_cls_jsc_2024-03-20/optimized/version_14/model.onnx
INFO     ONNX Model Summary: 
+-------+----------------------------------+--------------------+--------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+---------------------+
| 

We can view a summary of the ONNX model (which is the unmodified from the Pytorch one), however it should be optimized. Let's run an analysis path on both the original `MaseGraph` and the `.onnx` optimized model.

In [16]:
mg, _ = runtime_analysis_pass(mg, pass_args=runtime_analysis_config)

INFO     Starting transformation analysis on jsc-toy-onnx
I0320 18:14:11.922311 140225141540672 analysis.py:270] Starting transformation analysis on jsc-toy-onnx
INFO     
Results jsc-toy-onnx:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.70913    |
|      Average Precision       |    0.7384     |
|        Average Recall        |    0.72566    |
|       Average F1 Score       |    0.72853    |
|         Average Loss         |    0.76326    |
|       Average Latency        |  0.30757 ms   |
|   Average GPU Power Usage    |   22.334 W    |
| Inference Energy Consumption | 0.0019081 mWh |
+------------------------------+---------------+
I0320 18:14:12.557337 140225141540672 analysis.py:393] 
Results jsc-toy-onnx:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+----------

In [18]:
mg, _ = runtime_analysis_pass(onnx_meta['onnx_path'], pass_args=runtime_analysis_config)

INFO     Starting transformation analysis on jsc-toy-onnx
I0320 18:14:21.423943 140225141540672 analysis.py:270] Starting transformation analysis on jsc-toy-onnx
INFO     
Results jsc-toy-onnx:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+---------------+
|    Average Test Accuracy     |    0.70913    |
|      Average Precision       |    0.7384     |
|        Average Recall        |    0.72566    |
|       Average F1 Score       |    0.72853    |
|         Average Loss         |    0.76326    |
|       Average Latency        |  0.29346 ms   |
|   Average GPU Power Usage    |   22.203 W    |
| Inference Energy Consumption | 0.0018099 mWh |
+------------------------------+---------------+
I0320 18:14:22.097580 140225141540672 analysis.py:393] 
Results jsc-toy-onnx:
+------------------------------+---------------+
|            Metric            |     Value     |
+------------------------------+----------

As shown above, the latency has decreased around 4x with the `jsc-toy` model without compromising accuracy due to the well calibrated amax and quantization-aware fine tuning. The inference energy consumption has thus also dropped tremendously and this is an excellent demonstration for the need to quantize in industry especially for LLMs in order to reduce energy usage. 

## Section 2. INT8 Quantization

We may quantize either using FP16 or INT8 by setting the `precision` parameter in `passes.onnxruntime.default.config` to `'fp16'` or `'int8'` respectively. INT8 quantization will show the most notable latency improvements but will likely lower performance. 

There are two types of quantization for ONNXRT and can be set in `onnxruntime.default.config` under `quantization_types`. The differences are for how they calibrate i.e. set the scale and zero points which are only relevant for integer based quantization:
- **Static Quantization**:
    - The scale and zero point of activations are calculated in advance (offline) using a calibration data set.
    - The activations have the same scale and zero point during each forward pass.
- **Dynamic Quantization**:
    - The scale and zero point of activations are calculated on-the-fly (online) and are specific for each forward pass.
    - This approach is more accurate but introduces extra computational overhead.

Both methodolgies first pre-procsses the model before quantization adding further optimizations. This intermidate model is stored to the `pre-processed` directory. 

For this example, we will set the `precision` to `'int8'` and the `precision_types` to `['static', 'dynamic']` to compare both quantization methods, whilst keeping the other settings the exact same for a fair comparison against the optimized model. This time however, we will use chop from the terminal which runs the same pass.

In [6]:
JSC_TOML_PATH = "../../../machop/configs/onnx/jsc_gpu_quant.toml"

In [9]:
!ch transform --config {JSC_TOML_PATH}

[2024-03-20 19:02:29,496] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
INFO: Seed set to 0
I0320 19:02:31.689686 140260259174208 seed.py:54] Seed set to 0
+-------------------------+------------------------+--------------+-----------------+------------------------+
| Name                    |        Default         | Config. File | Manual Override |       Effective        |
+-------------------------+------------------------+--------------+-----------------+------------------------+
| task                    |     classification     |     cls      |                 |          cls           |
| load_name               |          None          |              |                 |          None          |
| load_type               |           mz           |              |                 |           mz           |
| batch_size              |          128           |     256      |                 |          256           |
| to_debug         